### Conversion Script for Criteo Dataset (CSV-to-Parquet) 

__Step 1__: Import libraries

In [1]:
import os
import rmm
import cudf
from cudf.io.parquet import ParquetWriter
from fsspec.core import get_fs_token_paths
import numpy as np
import pyarrow.parquet as pq

from dask.dataframe.io.parquet.utils import _analyze_paths
from dask.base import tokenize
from dask.utils import natural_sort_key
from dask.highlevelgraph import HighLevelGraph
from dask.delayed import Delayed
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

import nvtabular as nvt
from nvtabular.io import device_mem_size

/opt/conda/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/opt/conda/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice/.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


__Step 2__: Specify options

Specify the input and output paths, unless the `INPUT_DATA_DIR` and `OUTPUT_DATA_DIR` environment variables are already set. In order to utilize a multi-GPU system, be sure to specify `allow_multi_gpu=True` (and check the setting of your `CUDA_VISIBLE_DEVICES` environment variable).

In [2]:
INPUT_PATH = os.environ.get('INPUT_DATA_DIR', '/datasets/criteo/crit_orig')
OUTPUT_PATH = os.environ.get('OUTPUT_DATA_DIR', '/raid/criteo/tests/demo_out')
CUDA_VISIBLE_DEVICES = os.environ.get("CUDA_VISIBLE_DEVICES", "0")
n_workers = len(CUDA_VISIBLE_DEVICES.split(","))
frac_size = 0.15
allow_multi_gpu = False
use_rmm_pool = False
max_day = None  # (Optional) -- Limit the dataset to day 0-max_day for debugging

__Step 3__: Define helper/task functions

In [3]:
def _pool(frac=0.8):
    rmm.reinitialize(
        pool_allocator=True,
        initial_pool_size=frac * device_mem_size(),
    )
    
def _convert_file(path, name, out_dir, frac_size, fs, cols, dtypes):
    fn = f"{name}.parquet"
    out_path = fs.sep.join([out_dir, f"{name}.parquet"])
    writer = ParquetWriter(out_path, compression=None)
    for gdf in nvt.Dataset(
        path,
        engine="csv",
        names=cols,
        part_memory_fraction=frac_size,
        sep='\t',
        dtypes=dtypes,
    ).to_iter():
        writer.write_table(gdf)
        del gdf
    md = writer.close(metadata_file_path=fn)
    return md

def _write_metadata(md_list, fs, path):
    rg_sizes = []
    if md_list:
        metadata_path = fs.sep.join([path, "_metadata"])
        _meta = (
            cudf.io.merge_parquet_filemetadata(md_list)
            if len(md_list) > 1
            else md_list[0]
        )
        with fs.open(metadata_path, "wb") as fil:
            _meta.tofile(fil)
    return True

__Step 4__: (Optionally) Start a Dask cluster

In [4]:
# Start up cluster if we have multiple devices
# (and `allow_multi_gpu == True`)
client = None
if n_workers > 1 and allow_multi_gpu:
    cluster = LocalCUDACluster(
        n_workers=n_workers,
        CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
    )
    client = Client(cluster)
    if use_rmm_pool:
        client.run(_pool)
elif use_rmm_pool:
    _pool()

__Step 5__: Main conversion script (build Dask task graph)

In [5]:
fs = get_fs_token_paths(INPUT_PATH, mode="rb")[0]
file_list = [
    x for x in fs.glob(fs.sep.join([INPUT_PATH, "day_*"]))
    if not x.endswith("parquet")
]
file_list = sorted(file_list, key=natural_sort_key)
file_list = file_list[:max_day] if max_day else file_list
name_list = _analyze_paths(file_list, fs)[1]

cont_names = ["I" + str(x) for x in range(1, 14)]
cat_names = ["C" + str(x) for x in range(1, 27)]
cols = ["label"] + cont_names + cat_names

dtypes = {}
dtypes["label"] = np.int32
for x in cont_names:
    dtypes[x] = np.int32
for x in cat_names:
    dtypes[x] = "hex"

dsk = {}
token = tokenize(file_list, name_list, OUTPUT_PATH, frac_size, fs, cols, dtypes)
convert_file_name = "convert_file-" + token
for i, (path, name) in enumerate(zip(file_list, name_list)):
    key = (convert_file_name, i)
    dsk[key] = (_convert_file, path, name, OUTPUT_PATH, frac_size, fs, cols, dtypes)

write_meta_name = "write-metadata-" + token
dsk[write_meta_name] = (
    _write_metadata,
    [(convert_file_name, i) for i in range(len(file_list))],
    fs,
    OUTPUT_PATH,
)
graph = HighLevelGraph.from_collections(write_meta_name, dsk, dependencies=[])
conversion_delayed = Delayed(write_meta_name, graph)

**__Step 6__**: Execute conversion

In [6]:
%%time
if client:
    conversion_delayed.compute()
else:
    conversion_delayed.compute(scheduler="synchronous")

KeyboardInterrupt: 